<a href="https://colab.research.google.com/github/xhavien/CCMACLRL_EXERCISE/blob/main/Exercise9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercise 9: Choosing the best performing model on a dataset

Instructions:

- Use the Dataset File to train your model
- Use the Test File to generate your results
- Use the Sample Submission file to generate the same format
- Use all classification models

Submit your results to:
https://www.kaggle.com/competitions/playground-series-s4e10/overview



In [156]:
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

## Dataset File

In [157]:
dataset_url = 'https://github.com/robitussin/CCMACLRL_EXERCISES/blob/main/datasets/loan_approval/train.csv?raw=true'
df = pd.read_csv(dataset_url)

## Test File

In [158]:
test_url = 'https://github.com/robitussin/CCMACLRL_EXERCISES/blob/main/datasets/loan_approval/test.csv?raw=true'
dt=pd.read_csv(test_url)

In [159]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39098 entries, 0 to 39097
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          39098 non-null  int64  
 1   person_age                  39098 non-null  int64  
 2   person_income               39098 non-null  int64  
 3   person_home_ownership       39098 non-null  object 
 4   person_emp_length           39098 non-null  float64
 5   loan_intent                 39098 non-null  object 
 6   loan_grade                  39098 non-null  object 
 7   loan_amnt                   39098 non-null  int64  
 8   loan_int_rate               39098 non-null  float64
 9   loan_percent_income         39098 non-null  float64
 10  cb_person_default_on_file   39098 non-null  object 
 11  cb_person_cred_hist_length  39098 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 3.6+ MB


## Sample Submission File

In [210]:
sample_submission_url ='https://github.com/robitussin/CCMACLRL_EXERCISES/blob/main/datasets/loan_approval/sample_submission.csv?raw=true'

sf=pd.read_csv(sample_submission_url)

In [201]:
sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39098 entries, 0 to 39097
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           39098 non-null  int64  
 1   loan_status  39098 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 611.0 KB


In [162]:
df.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


In [163]:
from sklearn.model_selection import train_test_split

# Assuming 'loan_status' or a similar column is your actual target variable
y = df['loan_status']  # Replace 'loan_status' with the actual target column name
x = df.drop(['id', 'loan_status'], axis=1) # Remove target variable from features

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=0)

## 1. Train a KNN Classifier

In [164]:
score_list = {}

In [165]:
# Assuming 'loan_status' or a similar column is your actual target variable
y = df['loan_status']  # Replace 'loan_status' with the actual target column name
x = df.drop(['id', 'loan_status'], axis=1) # Remove target variable from features

# Identify categorical features (columns with object dtype)
categorical_features = x.select_dtypes(include=['object']).columns

# Create a OneHotEncoder instance
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore') # sparse=False for compatibility with KNN

# Fit the encoder on the categorical features and transform them
encoded_features = encoder.fit_transform(x[categorical_features])

# Create a DataFrame from the encoded features
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))

# Replace original categorical features with encoded features
x = x.drop(categorical_features, axis=1)
x = pd.concat([x, encoded_df], axis=1)

# Now you can proceed with train_test_split and model training
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=0)

score_list = {}

KNN = KNeighborsClassifier(n_neighbors=22) #I've tried more than 50 values. 22 is the best value

KNN.fit(x_train,y_train) # This should now work without the ValueError
knn_score = KNN.score(x_test,y_test)
score_list["KNN Classifier"] = knn_score
print(f"Score is {knn_score}")

Score is 0.8928297382556057


- Perform cross validation

In [166]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

# Initialize the KNN classifier
KNN = KNeighborsClassifier(n_neighbors=22)  # Your chosen best value

# Perform cross-validation
cv_scores = cross_val_score(KNN, x_train, y_train, cv=5)  # Change cv to your preferred number of folds

# Store the mean and standard deviation of the cross-validation scores
score_list = {
    "KNN Classifier Mean Score": np.mean(cv_scores),
    "KNN Classifier Std Dev": np.std(cv_scores)
}

# Fit the model on the entire training set for final evaluation
KNN.fit(x_train, y_train)
knn_score = KNN.score(x_test, y_test)

# Print the results
print(f"Cross-Validation Mean Score: {score_list['KNN Classifier Mean Score']:.4f}")
print(f"Cross-Validation Std Dev: {score_list['KNN Classifier Std Dev']:.4f}")
print(f"Test Score: {knn_score:.4f}")


Cross-Validation Mean Score: 0.8929
Cross-Validation Std Dev: 0.0025
Test Score: 0.8928


## 2. Train a Logistic Regression Classifier

In [167]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(x_train,y_train)

lr_score = LR.score(x_test,y_test)
score_list["Logistic Regression"] = lr_score

print(f"Score is {lr_score}")

Score is 0.877994713956859


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


- Perform cross validation

In [168]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Initialize the Logistic Regression model
LR = LogisticRegression()

# Perform cross-validation
cv_scores_lr = cross_val_score(LR, x_train, y_train, cv=5)  # Adjust cv for the number of folds as needed

# Store the mean and standard deviation of the cross-validation scores
score_list["Logistic Regression Mean Score"] = np.mean(cv_scores_lr)
score_list["Logistic Regression Std Dev"] = np.std(cv_scores_lr)

# Fit the model on the entire training set for final evaluation
LR.fit(x_train, y_train)
lr_score = LR.score(x_test, y_test)

# Print the results
print(f"Cross-Validation Mean Score for Logistic Regression: {score_list['Logistic Regression Mean Score']:.4f}")
print(f"Cross-Validation Std Dev for Logistic Regression: {score_list['Logistic Regression Std Dev']:.4f}")
print(f"Test Score for Logistic Regression: {lr_score:.4f}")


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Cross-Validation Mean Score for Logistic Regression: 0.8835
Cross-Validation Std Dev for Logistic Regression: 0.0026
Test Score for Logistic Regression: 0.8780


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## 3. Train a Naive Bayes Classifier

In [169]:
from sklearn.naive_bayes import GaussianNB

nbc = GaussianNB()
nbc.fit(x_train,y_train)
nbc_score = nbc.score(x_test,y_test)
score_list["GaussianNBC"] = nbc_score

print(f"Score is {nbc_score}")

Score is 0.8808935118083383


- Perform cross validation

In [170]:
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

# Initialize the Gaussian Naive Bayes model
nbc = GaussianNB()

# Perform cross-validation
cv_scores_nbc = cross_val_score(nbc, x_train, y_train, cv=5)  # Adjust cv for the number of folds as needed

# Store the mean and standard deviation of the cross-validation scores
score_list["GaussianNBC Mean Score"] = np.mean(cv_scores_nbc)
score_list["GaussianNBC Std Dev"] = np.std(cv_scores_nbc)

# Fit the model on the entire training set for final evaluation
nbc.fit(x_train, y_train)
nbc_score = nbc.score(x_test, y_test)

# Print the results
print(f"Cross-Validation Mean Score for Gaussian NBC: {score_list['GaussianNBC Mean Score']:.4f}")
print(f"Cross-Validation Std Dev for Gaussian NBC: {score_list['GaussianNBC Std Dev']:.4f}")
print(f"Test Score for Gaussian NBC: {nbc_score:.4f}")


Cross-Validation Mean Score for Gaussian NBC: 0.8836
Cross-Validation Std Dev for Gaussian NBC: 0.0013
Test Score for Gaussian NBC: 0.8809


## 4. Train a SVM Classifier

In [171]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(x_train,y_train)
svc_score = svc.score(x_test,y_test)
score_list["SVC"] = svc_score
print(f"Score is {svc_score}")

Score is 0.8576178702361668


- Perform cross validation

In [172]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

# Initialize the Support Vector Classifier
svc = SVC()

# Perform cross-validation
cv_scores_svc = cross_val_score(svc, x_train, y_train, cv=5)  # Adjust cv for the number of folds as needed

# Store the mean and standard deviation of the cross-validation scores
score_list["SVC Mean Score"] = np.mean(cv_scores_svc)
score_list["SVC Std Dev"] = np.std(cv_scores_svc)

# Fit the model on the entire training set for final evaluation
svc.fit(x_train, y_train)
svc_score = svc.score(x_test, y_test)

# Print the results
print(f"Cross-Validation Mean Score for SVC: {score_list['SVC Mean Score']:.4f}")
print(f"Cross-Validation Std Dev for SVC: {score_list['SVC Std Dev']:.4f}")
print(f"Test Score for SVC: {svc_score:.4f}")


Cross-Validation Mean Score for SVC: 0.8576
Cross-Validation Std Dev for SVC: 0.0000
Test Score for SVC: 0.8576


## 5. Train a Decision Tree Classifier

In [173]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=1)
dtc.fit(x_train,y_train)

dtc_score = dtc.score(x_test,y_test)
score_list["DTC"] = dtc_score
print(f"Score is {dtc_score}")

Score is 0.9132065819762981


- Perform cross validation

In [174]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

# Initialize the Decision Tree Classifier
dtc = DecisionTreeClassifier(random_state=1)

# Perform cross-validation
cv_scores_dtc = cross_val_score(dtc, x_train, y_train, cv=5)  # Adjust cv for the number of folds as needed

# Store the mean and standard deviation of the cross-validation scores
score_list["DTC Mean Score"] = np.mean(cv_scores_dtc)
score_list["DTC Std Dev"] = np.std(cv_scores_dtc)

# Fit the model on the entire training set for final evaluation
dtc.fit(x_train, y_train)
dtc_score = dtc.score(x_test, y_test)

# Print the results
print(f"Cross-Validation Mean Score for DTC: {score_list['DTC Mean Score']:.4f}")
print(f"Cross-Validation Std Dev for DTC: {score_list['DTC Std Dev']:.4f}")
print(f"Test Score for DTC: {dtc_score:.4f}")


Cross-Validation Mean Score for DTC: 0.9131
Cross-Validation Std Dev for DTC: 0.0033
Test Score for DTC: 0.9132


## 6. Train a Random Forest Classifier

In [175]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=50,random_state=1)
rfc.fit(x_train,y_train)
rfc_score = rfc.score(x_test,y_test)
score_list["RFC"]=rfc_score

print(f"Score is {rfc_score}")

Score is 0.95046466024384


## 7. Compare all the performance of all classification models

In [176]:

score_list = list(score_list.items())

In [177]:
for alg,score in score_list:
    print(f"{alg} Score is {str(score)[:4]} ")

KNN Classifier Mean Score Score is 0.89 
KNN Classifier Std Dev Score is 0.00 
Logistic Regression Score is 0.87 
Logistic Regression Mean Score Score is 0.88 
Logistic Regression Std Dev Score is 0.00 
GaussianNBC Score is 0.88 
GaussianNBC Mean Score Score is 0.88 
GaussianNBC Std Dev Score is 0.00 
SVC Score is 0.85 
SVC Mean Score Score is 0.85 
SVC Std Dev Score is 6.06 
DTC Score is 0.91 
DTC Mean Score Score is 0.91 
DTC Std Dev Score is 0.00 
RFC Score is 0.95 


## 9. Generate Submission File

Choose the model that has the best performance to generate a submission file.

In [188]:
dt.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,58645,23,69000,RENT,3.0,HOMEIMPROVEMENT,F,25000,15.76,0.36,N,2
1,58646,26,96000,MORTGAGE,6.0,PERSONAL,C,10000,12.68,0.10,Y,4
2,58647,26,30000,RENT,5.0,VENTURE,E,4000,17.19,0.13,Y,2
3,58648,33,50000,RENT,4.0,DEBTCONSOLIDATION,A,7000,8.90,0.14,N,7
4,58649,26,102000,MORTGAGE,8.0,HOMEIMPROVEMENT,D,15000,16.32,0.15,Y,4


In [206]:
x = dt.drop(['id'], axis=1) # Remove target variable from features

# Identify categorical features (columns with object dtype)
categorical_features = x.select_dtypes(include=['object']).columns

# Create a OneHotEncoder instance
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore') # sparse=False for compatibility with KNN

# Fit the encoder on the categorical features and transform them
encoded_features = encoder.fit_transform(x[categorical_features])

# Create a DataFrame from the encoded features
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))

# Replace original categorical features with encoded features
x = x.drop(categorical_features, axis=1)
x = pd.concat([x, encoded_df], axis=1)

In [208]:
dt = x

In [211]:
id = sf.pop('id')
y_pred = rfc.predict(dt)

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'id': id,
    'loan_status': y_pred
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission_file.csv', index=False)
print("Submission file created: submission_file.csv")

Submission file created: submission_file.csv
